  Copyright 2022 Google LLC.

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

In [ ]:
# @title Install required packages
def install_required_packages():
  global gym_minigrid
  global mon_minigrid
  global mdp_wrapper
  global coloring_wrapper
  !pip install dm-haiku
  !pip install flax

  !pip install gym_minigrid
  !git clone https://github.com/google-research/google-research.git

  GOOGLE_RESEARCH_PATH = './google-research'
  import sys
  if GOOGLE_RESEARCH_PATH not in sys.path:
    sys.path.append(GOOGLE_RESEARCH_PATH)

  from minigrid_basics.envs import mon_minigrid
  from minigrid_basics.custom_wrappers import mdp_wrapper
  from minigrid_basics.custom_wrappers import coloring_wrapper

install_required_packages()

In [ ]:
#@title Imports
import numpy as np
import scipy as sp

import jax
import jax.numpy as jnp
import jax.scipy as jsp

import matplotlib.pylab as plt
import numpy as np
import math
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import gin
import matplotlib.pyplot as plt
import numpy as np
import pdb
import sklearn
import tensorflow.compat.v1 as tf
import gym
import pdb

import jax.numpy as jnp
import jax
from jax import random
from jax import grad, jit
from flax import linen as nn
import collections
import numpy as np
import matplotlib.pylab as plt
import jax.numpy as jnp
import jax
import optax
import functools
import time
import pdb
import scipy

In [ ]:
# @title Losses

def inner_objective_td(Phi, G, gamma, P, W):
  return jnp.sum(jnp.square(Phi @ W - G - jax.lax.stop_gradient(gamma * P @ Phi @ W)))

def outer_objective_td(Phi, G, gamma, P):
  # Outer Objective function: $ \min_W \|\Phi W - sg(G - gamma P Phi W) \|^2_F$
  W_star, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ G)
  return inner_objective_td(Phi, G, gamma, P, W_star)
  
def mspbe(Phi, G, gamma, P, W):
  return jnp.linalg.norm(Phi @ W - Phi @ jnp.linalg.solve(Phi.T @ Phi, jnp.eye(Phi.shape[1])) @ Phi.T @ (G + gamma * P @ Phi @ W), ord='fro')**2

def outer_mspbe(Phi, G, gamma, P):
  W, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ G)
  return jnp.linalg.norm(Phi @ W - Phi @ jnp.linalg.solve(Phi.T @ Phi, jnp.eye(Phi.shape[1])) @ Phi.T @ (G + gamma * P @ Phi @ W), ord='fro')**2

def mspbe_nonlinear(params, Phi, G, gamma, P, W):
  return jnp.linalg.norm(params @ jnp.linalg.solve(params.T @ params, jnp.eye(params.shape[1])) @ params.T @ (G + gamma * P @ Phi @ W - Phi @ W), ord='fro')**2

def msbe_phi(Phi, G, gamma, P, W):
  return jnp.linalg.norm(Phi @ W - G @ W.T @ jnp.linalg.solve(W @ W.T, jnp.eye(Phi.shape[1])) @ W - gamma * P @ Phi @ W, ord='fro')**2

def outer_mspbe_Phi(Phi, G, gamma, P):
   W, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ G)
   return jnp.linalg.norm(Phi @ W - G @ W.T @ jnp.linalg.solve(W @ W.T, jnp.eye(Phi.shape[1])) @ W - gamma * P @ Phi @ W, ord='fro')**2
  

def outer_objective_mspbe(Phi, G, gamma, P, W):
  W_star, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ G)
  return inner_objective_td_true(Phi, G, gamma, P, W_star)

def policy_evaluation_error_td(Phi, G, gamma, P):
  W_star, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ G)
  return jnp.linalg.norm(Phi @ W_star - G - gamma * P @ Phi @ W_star, ord='fro')**2

def inner_objective_res(Phi, G, gamma, P, W):
  # Inner Objective function: $\|\Phi W - (G - gamma P Phi W) \|^2_F$
  return jnp.linalg.norm(Phi @ W - G - gamma * P @ Phi @ W, ord='fro')**2

def outer_objective_res(Phi, G, gamma, P):
  # Outer Objective function: $ \min_W \|\Phi W - (G - gamma P Phi W) \|^2_F$
  W_star, _, _, _ = jnp.linalg.lstsq((Phi - gamma * P @ Phi), G)
  return inner_objective_res(Phi, G, gamma, P, W_star)

def inner_objective_mc(Phi, Psi, W):
  # Inner Objective function: $\|\Phi W - \Psi \|^2_F$
  return jnp.linalg.norm(Phi @ W - Psi, ord='fro')**2

def outer_objective_mc(Phi, Psi):
  # Outer objective function: $J(\Phi) =\min_W \|\Phi W - \Psi \|^2_F$
  W_star, _, _, _ = jnp.linalg.lstsq(Phi, Psi, rcond=1e-5)
  return inner_objective_mc(Phi, Psi, W_star)

def grassman_distance(Y1, Y2):
  Q1, _ = jnp.linalg.qr(Y1)
  Q2, _ = jnp.linalg.qr(Y2)
  _, sigma, _ = jnp.linalg.svd(Q1.T @ Q2)
  # sigma = jnp.clip(sigma, -1., 1.)
  sigma = jnp.round(sigma, decimals=5)
  # pdb.set_trace()
  return jnp.linalg.norm(jnp.arccos(sigma))


grad_Phi_td = jax.grad(inner_objective_td, argnums=0)
grad_W_td = jax.grad(inner_objective_td, argnums=4)
grad_W_Phi_td = jax.grad(inner_objective_td, argnums=(0, 4))

grad_Phi_res = jax.grad(inner_objective_res, argnums=0)
grad_W_res = jax.grad(inner_objective_res, argnums=4)
grad_W_Phi_res = jax.grad(inner_objective_res, argnums=(0, 4))

grad_Phi_mc = jax.grad(inner_objective_mc, argnums=0) # shape (S, d)
grad_W_mc = jax.grad(inner_objective_mc, argnums=2) # shape (d, T)
grad_W_Phi_mc = jax.grad(inner_objective_mc, argnums=(0, 2)) # tuple shape (S, d) and (d, T)

In [ ]:
S = 2
T = S
d = 2

from jax.config import config
config.update("jax_enable_x64", True)

gamma = 0.99
Phi = random.normal(random.PRNGKey(0), (S, d), dtype=jnp.float64)
Phi = Phi / jnp.linalg.norm(Phi)
W = random.normal(random.PRNGKey(10), (d, 1), dtype=jnp.float64)
W = W / jnp.linalg.norm(W)
W_tile = jnp.tile(W, (1, S))
params = jnp.concatenate((W_tile.T, Phi), axis=1)

G = random.normal(random.PRNGKey(0), (S, T), dtype=jnp.float64)
P = jax.random.normal(random.PRNGKey(0), (S, S), dtype=jnp.float64) #+ jax.random.normal(random.PRNGKey(0), (S, S), dtype=jnp.float64).T
P /= jnp.sum(P, axis=1)

W_star_td = sp.linalg.solve(Phi.T @ (Phi - gamma * P @ jax.lax.stop_gradient(Phi)), jnp.eye(d)) @ Phi.T @ G
mspbe(Phi, G, gamma, P, W_star_td)
jnp.linalg.solve(params.T @ params, jnp.eye(params.shape[1]))
print(params @ jnp.linalg.solve(params.T @ params, jnp.eye(2*d)) @ params.T)
P_perp = (jnp.eye(S) - W_tile.T @ (jnp.linalg.solve(W_tile @ W_tile.T, jnp.eye(d)))@W_tile)
print(P_perp @ Phi @ jnp.linalg.solve(Phi.T @ P_perp @ Phi, jnp.eye(d)) @ Phi.T @ (-P_perp) + jnp.eye(S))
jax.grad(mspbe_nonlinear, argnums=(0))(params, Phi, G, gamma, P, W), jax.grad(inner_objective_td, argnums=(0, 4))(Phi, G, gamma, P, W)
A = W_tile @ W_tile.T
B = W_tile @ Phi
C = Phi.T @ W_tile.T
D = Phi.T @ Phi
A_inv = jnp.linalg.solve(A, jnp.eye(d))
inv_term = jnp.linalg.solve(D - C @ A_inv @ B, jnp.eye(d))
M_D = A = B@ jnp.linalg.solve(D, jnp.eye(d)) @ C
M_D

In [ ]:
print(jnp.linalg.solve(params.T @ params, jnp.eye(params.shape[1])))
jnp.linalg.matrix_rank(W_tile), jnp.linalg.matrix_rank(Phi)
# jnp.linalg.solve(params.T @ params, jnp.eye(2*d))
W @ W.T, Phi.T @ Phi, params.T @ params

In [ ]:
W = random.normal(random.PRNGKey(0), (d, T), dtype=jnp.float64) * 0
Phi_star = random.normal(random.PRNGKey(0), (S, d), dtype=jnp.float64) * 0 #jnp.linalg.solve(jnp.eye(S) - gamma * P, jnp.eye(S)) @ G @ W.T @ jnp.linalg.solve(W @ W.T, jnp.eye(d))
msbe_phi(Phi_star, G, gamma, P, W)
# jnp.linalg.norm((Phi_star @ W).T - W.T @ jnp.linalg.solve(W @ W.T, jnp.eye(Phi_star.shape[1])) @ W @ G.T - gamma * (Phi_star @ W).T @ P.T, ord='fro')**2
msbe_phi(Phi_star, G, gamma, P, W)
# msbe_phi(jnp.linalg.solve(jnp.eye(S) - gamma * P, jnp.eye(S)) @ G / W, G, gamma, P, W)
jnp.linalg.norm(Phi_star @ W @ W.T - gamma * P @ Phi_star @ W @ W.T - G @ W.T, ord='fro')**2

In [ ]:
def f(Phi, W, i):
  return (Phi @ W)[i][0]

f(Phi, W, 0)
params = (Phi, W)
jax.grad(f, argnums=(0, 1))(Phi, W, 0) , W, Phi

In [ ]:
# @title RL basics
def policy_random(env):
  """Random policy on env.

  Args:
    env: a MiniGrid environment, including the MDPWrapper.

  Returns:
    Numpy array S \times A: random policy
  """
  return np.ones((env.num_states, env.num_actions)) / env.num_actions

def trans_matrix(env, policy):
  trans_matrix = np.zeros((env.num_states, env.num_states))
  # The transition probability is S x A x S. We need to make it S x S here.
  for s in range(env.num_states):
    for a in range(env.num_actions):
      for s_next in range(env.num_states):
        trans_matrix[s][s_next] += env.transition_probs[s][a][s_next] * policy[s][a]
  return trans_matrix

def get_state_xy(idx, num_cols):
  """
  Given the index that uniquely identifies each state this method returns its equivalent coordinate (x,y).

  :param idx: index uniquely identifying a state
  :return: values x, y describing the state's location in the grid
  """
  y = int(idx % num_cols)
  x = int((idx - y) / num_cols)
  return x, y


def get_state_idx(x, y, num_cols):
    """
    Given a state coordinate (x,y) this method returns the index that uniquely identifies this state.

    :param x: value of the coordinate x
    :param y: value of the coordinate y
    :return : unique index identifying a position in the grid
    """
    idx = y + x * num_cols
    return idx

In [ ]:
# @title Various MDPs

# Disconnected MDP
def P_disconnected(n):
  return np.eye(n)

# Baird's MDP aka "waterfall MDP"
def P_star_terminal(n):
  mat = np.zeros((n, n))
  mat[:, -1] = 1
  return mat

# Baird's MDP aka "waterfall MDP" with non terminal state n
def P_star_non_terminal(n):
  mat = np.zeros((n, n))
  mat[:n-1, -1] = 1
  mat[n-1, :n-1] = 1 / (n-1)
  return mat

# Random walk on discrete torus MDP = Cycle (mentioned in https://arxiv.org/pdf/2101.07123.pdf)
def P_torus(n):
  assert n >= 3
  c = np.zeros((n,))
  c[1] = 0.5
  c[-1] = 0.5
  return sp.linalg.circulant(c)

def circle(n):
  assert n >= 3
  c = np.zeros((n,))
  c[-1] = 1.
  return sp.linalg.circulant(c)  

# Fully connected MDP
def P_fullyconnected(n):
  assert n >= 2
  return (np.ones((n, n)) - np.eye(n)) / (n - 1)

# Linear Chain
def P_chain(n):
  P = P_torus(n)
  P[0, 0] = 0.5
  P[0, n-1] = 0.
  P[n-1, 0] = 0.
  P[n-1, n-1] = 0.5   
  return P

# Lattice MDP: square grid w/ x-coordinates being in the range 1, ..., 10 and y-coordinates being in the range 1, ..., 10
lattice = mon_minigrid.MonMiniGridEnv(
    ascii_grid= """
**********************
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
*                    *
**********************""",
directional = True,
agent_pos = None,
goal_pos = None)
lattice = mdp_wrapper.MDPWrapper(lattice)

P_lattice = trans_matrix(lattice, policy_random(lattice))

small_lattice = mon_minigrid.MonMiniGridEnv(
    ascii_grid= """
************
*          *
*          *
*          *
*          *
*          *
*          *
*          *
*          *
*          *
*          *
************""",
directional = True,
agent_pos = None,
goal_pos = None)
small_lattice = mdp_wrapper.MDPWrapper(small_lattice)

P_small_lattice = trans_matrix(small_lattice, policy_random(lattice))

# 2d cycle
# for every (i, j) the neighbors are just (i + 1 mod n, j), (i - 1 mod n, j), (i, j+1 mod n), (i, j-1 mod n)
# x-coordinates being in the range 1, ..., n and y-coordinates being in the range 1, ..., n
def P_torus_2d(num_states):
  n = int(np.sqrt(num_states))
  mat = np.zeros((num_states, num_states))
  for idx in range(num_states):
    x, y = get_state_xy(idx, n)
    nn1 = get_state_idx((x+1)%n, y, n)
    nn2 = get_state_idx((x-1)%n, y, n)
    nn3 = get_state_idx(x, (y+1)%n, n)
    nn4 = get_state_idx(x, (y-1)%n, n)
    mat[idx, nn1] = 0.25
    mat[idx, nn2] = 0.25
    mat[idx, nn3] = 0.25
    mat[idx, nn4] = 0.25
  return mat

def assert_square(F):
  assert len(F.shape) == 2
  assert F.shape[0] == F.shape[1]

def assert_transition_matrix(P):
  assert_square(P)
  assert (P >= 0).all()
  assert (P <= 1).all()
  assert np.allclose(P.sum(axis=1), np.ones((P.shape[0],)))

In [ ]:
def approx_error(Phi, v):
  S, _ = Phi.shape
  k = np.linalg.matrix_rank(Phi)
  if test_orthogonal(Phi):
    F_k = Phi
    P_perp_term = (np.eye(S) - F_k@F_k.T) @ v
    approx_error = jnp.linalg.norm(P_perp_term, ord='fro')**2
    return approx_error
  else:
    F, _, _ = jnp.linalg.svd(Phi)
    F_k = F[:, :k]
    P_perp_term = (np.eye(S) - F_k@F_k.T) @ v
    approx_error = jnp.linalg.norm(P_perp_term, ord='fro')**2
    return approx_error

def test_orthogonal(F):
  return np.allclose(F.T @ F, np.eye(F.shape[1]), atol=1e-6)

In [ ]:
def lstd_solution(Phi, P, nu, r, gamma):
  # infinite sample lastd solution
  N = nu * np.eye(S)
  right = Phi.T @ N @ r
  left = Phi.T @ N @ (np.eye(S) - gamma * P) @ Phi 
  x, _, _, _ = np.linalg.lstsq(left, right, rcond=1e-5)
  return x


def bellman_variance(P, r, Phi, gamma, nu, num_iterations=1000):
  # empirical estimation of the variance of the bellman residual
  # nu is the stationary distribution
  S, d = Phi.shape
  L = []
  N = nu * np.eye(S)
  right = Phi.T @ N @ r
  left = Phi.T @ N @ (np.eye(S) - gamma * P)
  # pdb.set_trace()
  V, _, _, _ = np.linalg.lstsq(left, right, rcond=1e-5)
  for i in range (num_iterations):
    # s is drawn according to the stationary distribution
    u = np.random.rand(1, P.shape[0])
    cdf_s = np.cumsum(nu)
    s = (u < cdf_s).argmax()
    # s' is drawn according to the distribution incuced by P over the states
    u_prime = np.random.rand(1, P.shape[0])
    cdf_s_prime = np.cumsum(P[s])
    s_prime = (u_prime < cdf_s).argmax()
    # pdb.set_trace()
    L.append((V[s] - r[s] - gamma * V[s_prime])**2)
  return sum(L) / len(L)

def stationary_distribution(P):
  vals , v = np.linalg.eig(P.T)
  vals = np.round(vals, decimals=6)
  idx = np.where(np.round(vals, decimals=6) == 1.)[0][0]
  return v[:, idx] / np.sum(v[:, idx])

def rho_0(Phi, dis):
  """
  Args:
    Phi: S times d matrix
  """
  S, d = Phi.shape
  N = dis * np.eye(S)
  Xi = Phi.T @ N @ Phi
  return np.max(np.sum(np.square(Phi @ np.linalg.inv(scipy.linalg.sqrtm(Xi))), axis=1))

In [ ]:
S = 123
gamma = 0.99
P = P_torus(S)
assert_transition_matrix(P)
Phi = sp.linalg.solve(np.eye(S) - gamma * P, np.eye(S))
#have stationary distributions: P star terminal, disconnected, torus, fully connected, chain, lattice, 2dtorus
nu = stationary_distribution(P)
r = np.ones(S)
Phi @ lstd_solution(Phi, P, nu, r, gamma), Phi @ r
bellman_variance(P, r, Phi, gamma, nu)

In [ ]:
np.round((P**100)[0], decimals=6) == np.round((P**100)[0] @ P, decimals=6)
# lamb, v = np.linalg.eig(P.T)
# (0.45 < np.cumsum(stationary_distribution(P)))
# np.round(P.T @ v[:, 2], decimals=6) == np.round(v[:, 2], decimals=6)

In [ ]:
l = {'Disconnected': P_disconnected, 'Chain': P_chain, 'Fully connected': P_fullyconnected, 'Star': P_star_terminal}
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(5*4, 8))
i = 0
for (key, p) in l.items():
  print(key)
  ax = axes[:, i]
  i +=1
  s_range = range(3, 400)
  var_range_all_ones = []
  var_range_one_hot = []
  var_range_gauss = []
  for S in s_range:
    P = p(S)
    assert_transition_matrix(P)
    Phi = sp.linalg.solve(np.eye(S) - gamma * P, np.eye(S))
    r_all_ones = np.ones((S,))
    r_one_hot = np.zeros((S,))
    r_one_hot[-1] = 1.0
    r_gauss = np.random.normal(size=S)
    r_gauss /= np.linalg.norm(r_gauss, ord=np.inf) # normalize to R_max=1

    nu = stationary_distribution(P)
    var_range_all_ones.append(bellman_variance(P, r_all_ones, Phi, gamma, nu))
    var_range_one_hot.append(bellman_variance(P, r_one_hot, Phi, gamma, nu))
    var_range_gauss.append(bellman_variance(P, r_gauss, Phi, gamma, nu))

  ax[0].plot(s_range, var_range_all_ones)
  ax[1].plot(s_range, var_range_one_hot)
  # plt.plot(s_range, var_range_gauss)
  ax[0].legend(['all_ones'])
  ax[1].legend(['one_hot'])
  ax[0].set_title(key, size='x-large')
  ax[0].set_xlabel('Number of states', size='x-large') 
  ax[1].set_ylabel('Bellman residual variance', size='x-large')
  ax[1].set_xlabel('Number of states', size='x-large') 
  ax[0].set_ylabel('Bellman residual variance', size='x-large')


In [ ]:
S = 400
gamma = 0.99
l = {'Chain': P_chain, 'Fully connected': P_fullyconnected, 'Star': P_star_terminal, 'torus': P_torus, '2dtorus': P_torus_2d, 'Lattice': P_lattice}

fig, axes = plt.subplots(ncols=6, nrows=1, figsize=(5*6, 4))
i = 0
for (key, p) in l.items():
  if isinstance(p, np.ndarray):
    pass
  else:
    P = p(S)
  print(key)
  # ax = axes[:, i]
  ax = axes[i]
  i +=1
  k_range = range(3, S)
  approx_range_sl = []
  approx_range_res = []
  approx_range_td = []
  _, Z = scipy.linalg.schur(P)
  assert_transition_matrix(P)
  Psi = sp.linalg.solve(np.eye(S) - gamma * P, np.eye(S))
  F, s, B = np.linalg.svd(Psi)
  G = jnp.eye(S)
  G_S, sigma_g, _ = jnp.linalg.svd(G)
  r_all_ones = np.ones((S,))
  r_one_hot = np.zeros((S,))
  r_one_hot[-1] = 1.0
  r_gauss = np.random.normal(size=S)
  r_gauss /= np.linalg.norm(r_gauss, ord=np.inf) # normalize to R_max=1
  v_all_ones = (Psi @ r_all_ones).reshape(S, 1)
  v_one_hot = (Psi @ r_one_hot).reshape(S, 1)
  v_gauss = (Psi @ r_gauss).reshape(S, 1)

  for k in k_range:
    approx_range_sl.append(approx_error(F[:, :k], v_one_hot))
    approx_range_res.append(approx_error(Psi @ G_S[:, :k], v_one_hot))
    approx_range_td.append(approx_error(Z[:, :k], v_one_hot))

  ax.plot(k_range, approx_range_sl)
  ax.plot(k_range, approx_range_res)
  ax.plot(k_range, approx_range_td)
  ax.legend(['sl' , 'res', 'td'])
  ax.set_title(key, size='x-large')
  ax.set_xlabel('Number of features', size='x-large') 
  ax.set_ylabel('Approx error', size='x-large')
plt.savefig('all_ones_reward.png')
%download_file all_ones_reward.png

In [ ]:
# @title Learning different representations with SR auxiliary tasks 
gamma = 0.99
num_epochs = 1000
decay = 0.
start_learning_rate = 1e-4
optimizer = optax.sgd(start_learning_rate)
P = circle(9)
S = P.shape[0]
L = (np.eye(S) - gamma* P)
L_inv = np.linalg.solve(L, np.eye(S))

# G = jnp.eye(S, dtype=jnp.float64)
G = jnp.zeros(S).reshape(S, 1)
G = G.at[0].set(1)

T = S
d = 4

def train_td_representation(P, num_epochs, start_learning_rate, d):
  Phi_init, W_init = random.normal(random.PRNGKey(0), (S, d)), random.normal(random.PRNGKey(0), (d, S))
  params = (Phi_init, W_init)
  opt_state = optimizer.init(params)
  inner_objective_td(params[0], G, gamma, P, params[1])
  L = []
  for i in range(num_epochs):
    # alpha = 1 / (1 + decay * i) 
    grads = grad_W_Phi_td(params[0], G, gamma, P, params[1])
    updates, opt_state = optimizer.update(grads, opt_state)
    # updates = (alpha * updates[0], alpha * updates[1])
    params = optax.apply_updates(params, updates)
    L.append(params)
  return L

In [ ]:
# @title TD learning w/ outer objective
gamma = 0.99
num_epochs = 5000
decay = 0.
start_learning_rate = 1e-3
optimizer = optax.sgd(start_learning_rate)
P = circle(9)
S = P.shape[0]

L = (np.eye(S) - gamma* P)
L_inv = np.linalg.solve(L, np.eye(S))

# G = jnp.eye(S, dtype=jnp.float64)
G = jnp.zeros(S).reshape(S, 1)
G = G.at[0].set(1)

T = S
d = 2

def train_td_representation_outer(P, num_epochs, start_learning_rate, d):
  Phi_init = random.normal(random.PRNGKey(0), (S, d))
  params = Phi_init
  opt_state = optimizer.init(params)
  outer_objective_td(params, G, gamma, P)
  L = []
  for i in range(num_epochs):
    # alpha = 1 / (1 + decay * i) 
    alpha =1
    grads = jax.grad(outer_objective_td, argnums=0)(params, G, gamma, P)
    updates, opt_state = optimizer.update(grads, opt_state)
    # updates = (alpha * updates[0], alpha * updates[1])
    params = optax.apply_updates(params, alpha * updates)
    L.append(params)
  return L

In [ ]:
L = train_td_representation_outer(P, num_epochs, start_learning_rate, d)

In [ ]:
y = [outer_objective_td(k, G, gamma, P) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
data = np.zeros(num_epochs)
plt.plot(np.arange(1, num_epochs+1), data)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
plt.ylim(bottom=0)
%download_file image.png

In [ ]:
y = [(k[:, 1].T @ (L_inv @ G) / (np.linalg.norm(k[:, 1]) * np.linalg.norm(L_inv @ G)))[0] for k in L]
plt.plot(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
y = [max(abs((k / np.linalg.norm(k)) - ((L_inv @ G) / (np.linalg.norm(L_inv @ G))))) for k in L]
plt.plot(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
min((L[-1] / np.linalg.norm(L[-1])) - ((L_inv @ G) / (np.linalg.norm(L_inv @ G))))

In [ ]:
y = [inner_objective_td(k[0], G, gamma, P, k[1]) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
data = np.zeros(num_epochs)
plt.plot(np.arange(1, num_epochs+1), data)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
y = [mspbe(k[0], G, gamma, P, k[1]) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
y = [outer_mspbe(k, G, gamma, P) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
y = [msbe_phi(k[0], G, gamma, P, k[1]) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
(L[-1][0] @ L[-1][1]).shape

In [ ]:
y = [np.linalg.norm(k[0] @ k[1] - L_inv @ G, ord='frob')  for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
plt.xlabel('epochs')
plt.ylabel('frob norm')
plt.title('S={}, T={}, d={}, lr={}, decay={}'.format(S, T, d, start_learning_rate, decay))
plt.savefig('image.png')
%download_file image.png

In [ ]:
ranks = [np.linalg.matrix_rank(k) for k in L]
plt.plot(np.arange(1, num_epochs+1), ranks)

In [ ]:
jnp.linalg.solve(L[-1].T @ L[-1], jnp.eye(d) ), jnp.linalg.matrix_rank(L[-1].T @ L[-1])

In [ ]:
idx = -1
Phi = L[idx][0]
W = L[idx][1]
U, s, V = jnp.linalg.svd(jnp.linalg.inv(jnp.eye(S) - gamma* P))
Phi, (Phi.T @ (jnp.eye(S) - gamma * P) @ Phi), U, V
grad_W_Phi_td(Phi, G, gamma, P, W) # zero
# U @ Phi, U.T @ Phi, V.T @ Phi, V @ Phi
y = [grassman_distance(k[0], U[:, :d]) for k in L]
plt.semilogy(np.arange(1, num_epochs+1), y)
plt.ylim(bottom=0.0)
plt.xlabel('epochs')
plt.ylabel('grassman distance')

In [ ]:
Psi = sp.linalg.solve(np.eye(S) - gamma * P, np.eye(S))
F, s, B = np.linalg.svd(Psi)
r_all_ones = np.ones((S,))
r_one_hot = np.zeros((S,))
r_one_hot[-1] = 1.0
r_gauss = np.random.normal(size=S)
r_gauss /= np.linalg.norm(r_gauss, ord=np.inf) # normalize to R_max=1
v_all_ones = (Psi @ r_all_ones).reshape(S, 1)
v_one_hot = (Psi @ r_one_hot).reshape(S, 1)
v_gauss = (Psi @ r_gauss).reshape(S, 1)

In [ ]:
# TD representations
Phi = L[-1][0]
w_star_gauss, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_gauss)
w_star_all_ones, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_all_ones)
w_star_one_hot, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_one_hot)
jnp.sum((Phi @ w_star_gauss- v_gauss)**2), jnp.sum((Phi @ w_star_all_ones- v_all_ones)**2), jnp.sum((Phi @ w_star_one_hot- v_one_hot)**2)
max(jnp.sum((Phi @ w_star_gauss- v_gauss)**2), jnp.sum((Phi @ w_star_all_ones- v_all_ones)**2), jnp.sum((Phi @ w_star_one_hot- v_one_hot)**2))

In [ ]:
# Mc representations
Phi = F[:, :d]
w_star_gauss, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_gauss)
w_star_all_ones, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_all_ones)
w_star_one_hot, _, _, _ = jnp.linalg.lstsq(Phi.T @ (Phi - gamma * P @ Phi), Phi.T @ r_one_hot)
jnp.sum((Phi @ w_star_gauss- v_gauss)**2), jnp.sum((Phi @ w_star_all_ones- v_all_ones)**2), jnp.sum((Phi @ w_star_one_hot- v_one_hot)**2)
max(jnp.sum((Phi @ w_star_gauss- v_gauss)**2), jnp.sum((Phi @ w_star_all_ones- v_all_ones)**2), jnp.sum((Phi @ w_star_one_hot- v_one_hot)**2))

In [ ]:
S = 400
gamma = 0.99
# l = {'Chain': P_chain, 'Fully connected': P_fullyconnected, 'Star': P_star_terminal, 'torus': P_torus, '2dtorus': P_torus_2d, 'Lattice': P_lattice}
l = {'Lattice': P_lattice}
fig, axes = plt.subplots(ncols=6, nrows=1, figsize=(5*6, 4))
i = 0
for (key, p) in l.items():
  if isinstance(p, np.ndarray):
    pass
  else:
    P = p(S)
  print(key)
  # ax = axes[:, i]
  ax = axes[i]
  i +=1
  k_range = range(3, S, 50)
  approx_range_sl = []
  approx_range_td = []
  assert_transition_matrix(P)
  Psi = sp.linalg.solve(np.eye(S) - gamma * P, np.eye(S))
  F, s, B = np.linalg.svd(Psi)
  G = jnp.eye(S)
  r_all_ones = np.ones((S,))
  r_one_hot = np.zeros((S,))
  r_one_hot[-1] = 1.0
  r_gauss = np.random.normal(size=S)
  r_gauss /= np.linalg.norm(r_gauss, ord=np.inf) # normalize to R_max=1
  v_all_ones = (Psi @ r_all_ones).reshape(S, 1)
  v_one_hot = (Psi @ r_one_hot).reshape(S, 1)
  v_gauss = (Psi @ r_gauss).reshape(S, 1)

  for k in k_range:
    Phi_mc = F[:, :d]
    L = train_td_representation(P, 1000, 1e-2, k)
    approx_range_sl.append(())

  ax.plot(k_range, approx_range_sl)
  ax.plot(k_range, approx_range_td)
  ax.legend(['sl' , 'res', 'td'])
  ax.set_title(key, size='x-large')
  ax.set_xlabel('Number of features', size='x-large') 
  ax.set_ylabel('Approx error', size='x-large')
plt.savefig('all_ones_reward.png')
%download_file all_ones_reward.png